Python notebook based on https://towardsdatascience.com/fantasy-premier-league-value-analysis-python-tutorial-using-the-fpl-api-8031edfe9910#8ef2

Tutorial on how to get use the FPL API to gather data.

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
# API url
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

In [3]:
# Use requests to get request from API
request = requests.get(url)

In [4]:
json = request.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

Building the dataframes we will use

In [5]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

For loop that loops through phases dictionary to see what it contains

for x in range(0,len(json['phases'])):
    for key,value in json['phases'][x].items():
        print(key,value)

In [6]:
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,None,80201,0,0,0,0,0,1,3.5,...,326,4,196,12,NaN,,NaN,,NaN,
1,NaN,None,115918,0,0,0,0,0,1,2.0,...,339,13,355,31,NaN,,NaN,,NaN,
2,NaN,None,47431,0,0,0,0,0,3,3.2,...,146,85,124,71,1.0,,2.0,,NaN,
3,NaN,None,54694,0,0,0,0,0,4,4.2,...,32,16,50,18,NaN,,NaN,,1.0,
4,NaN,None,58822,0,0,0,0,0,2,2.7,...,264,86,293,100,NaN,,5.0,,NaN,


Looking at all the columns of our dataframe

In [7]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [8]:
slim_elements_df = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

In [9]:
slim_elements_df.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points
0,Leno,1,1,2.3,50,3131,0,26.2,131
1,Rúnarsson,1,1,1.1,40,15,0,0.2,1
2,Borges Da Silva,1,3,0.3,65,1398,0,12.0,78
3,Aubameyang,1,4,3.5,100,2330,0,13.1,131
4,Soares,1,2,0.3,45,744,0,6.2,28


In [10]:
slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [11]:
slim_elements_df.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position
0,Leno,1,1,2.3,50,3131,0,26.2,131,Goalkeeper
1,Rúnarsson,1,1,1.1,40,15,0,0.2,1,Goalkeeper
2,Borges Da Silva,1,3,0.3,65,1398,0,12.0,78,Midfielder
3,Aubameyang,1,4,3.5,100,2330,0,13.1,131,Forward
4,Soares,1,2,0.3,45,744,0,6.2,28,Defender


In [29]:
filter = slim_elements_df.second_name == 'Sancho'
slim_elements_df[filter]

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position
310,Sancho,13,3,5.9,95,0,0,0.0,0,Midfielder
